# Automated Forex Model Retraining Pipeline

This notebook:
1. Downloads latest forex data
2. Trains all 4 currency pair models
3. Compares with existing models
4. Pushes improved models to GitHub
5. Triggers HF Space deployment

**Schedule:** Run this weekly (Kaggle doesn't have native scheduling, but you can use external triggers)

## 1. Setup Environment

In [ ]:
# Clone your repository
!git clone https://github.com/Morriase/Macro-Technical-Sentiment-Classifier.git
%cd Macro-Technical-Sentiment-Classifier

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

In [ ]:
# Install TA-Lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
%cd ..
!pip install TA-Lib

## 2. Download Latest Data

In [ ]:
from src.data_acquisition.kaggle_loader import download_all_datasets

print("Downloading latest forex data...")
download_all_datasets()
print("✓ Data downloaded")

## 3. Backup Current Models

In [ ]:
import shutil
from pathlib import Path

# Backup existing models
if Path('models').exists():
    shutil.copytree('models', 'models_backup', dirs_exist_ok=True)
    print("✓ Models backed up")
else:
    print("No existing models to backup")

## 4. Train New Models

In [ ]:
# Train all 4 pairs
!python train_pipeline.py --pairs EUR_USD GBP_USD USD_JPY AUD_USD

## 5. Compare Model Performance

In [ ]:
!python scripts/compare_models.py --old models_backup --new models --output comparison.json

In [ ]:
import json

# Load comparison results
with open('comparison.json', 'r') as f:
    comparison = json.load(f)

print("\n" + "="*60)
print(f"DECISION: {'DEPLOY ✅' if comparison['should_deploy'] else 'SKIP ⏭️'}")
print(f"Reason: {comparison['summary']}")
print("="*60)

for result in comparison['results']:
    print(f"\n{result['pair']}:")
    print(f"  Improved: {result['improved']}")
    print(f"  Reason: {result['reason']}")

## 6. Deploy to GitHub (if improved)

In [ ]:
# Add your GitHub token as a Kaggle secret
# Go to: Kaggle Account → Settings → Add-ons → Secrets
# Add: GITHUB_TOKEN

from kaggle_secrets import UserSecretsClient
import os

if comparison['should_deploy']:
    print("Deploying improved models...")
    
    # Get GitHub token from Kaggle secrets
    user_secrets = UserSecretsClient()
    github_token = user_secrets.get_secret("GITHUB_TOKEN")
    
    # Configure git
    !git config user.name "Kaggle Bot"
    !git config user.email "kaggle@bot.com"
    
    # Add and commit new models
    !git add models/
    !git commit -m "Auto-retrain: Deploy improved models from Kaggle"
    
    # Push to GitHub using token
    repo_url = f"https://{github_token}@github.com/Morriase/Macro-Technical-Sentiment-Classifier.git"
    !git remote set-url origin {repo_url}
    !git push origin main
    
    print("✅ Models pushed to GitHub!")
    print("HF Space will auto-deploy in ~5 minutes")
else:
    print("⏭️ Skipping deployment - current models are better")

## 7. Trigger HF Space Rebuild (Optional)

In [ ]:
import requests

if comparison['should_deploy']:
    # Get HF token from Kaggle secrets
    hf_token = user_secrets.get_secret("HF_TOKEN")
    
    # Trigger rebuild
    response = requests.post(
        "https://huggingface.co/api/spaces/morriase/forex-live_server/restart",
        headers={"Authorization": f"Bearer {hf_token}"}
    )
    
    if response.status_code == 200:
        print("✅ HF Space rebuild triggered!")
    else:
        print(f"⚠️ HF rebuild failed: {response.status_code}")
        print("Space will auto-deploy from GitHub anyway")

## Summary

✅ Training complete  
✅ Models compared  
✅ Deployment decision made  
✅ GitHub updated (if improved)  
✅ HF Space will auto-deploy  

**Next run:** Schedule this notebook to run weekly